# Intraday Dynamic Trading Strategy (by deekseek)

## Strategy Details:

### Core Components:
Trend Identification (20-period EMA & 50-period EMA)

Volatility Measurement (Bollinger Bands 20,2)

Momentum Confirmation (RSI 14)

Range Detection (ATR 14)

Price Action Signals (Inside Bars & Fakeouts)

### Dual-Mode Entry System:

#### A. Trend Mode (Directional Bias):

Long Entry:

Price retraces to 20 EMA or lower Bollinger Band

RSI > 40 (maintains bullish momentum)

Volume > 1.5 * 20-period average

Stop Loss: Below recent swing low (1.5x ATR)

Short Entry:

Price rallies to 20 EMA or upper Bollinger Band

RSI < 60

Volume > 1.5 * 20-period average

Stop Loss: Above recent swing high (1.5x ATR)

#### B. Range Mode (Reversion Bias):

Long Entry:

Price touches lower Bollinger Band

RSI < 35 + bullish reversal candle

Stop Loss: 1 ATR below entry

Short Entry:

Price touches upper Bollinger Band

RSI > 65 + bearish reversal candle

Stop Loss: 1 ATR above entry

### Adaptive Exit Strategy:

pinescript:

Copy
// Profit Taking
trend_target = entry_price + (2.5 * atr(14)) 
range_target = entry_price + (1.2 * atr(14))

// Dynamic Trailing Stop
trail_stop = highest(high,3) - (1.2 * atr(14)) for longs
trail_stop = lowest(low,3) + (1.2 * atr(14)) for shorts

### Risk Management:

1-2% risk per trade

Position sizing based on stop distance

Max 3 concurrent positions

Time filter: Avoid first/last 30 minutes

### Trade Management:

Scale out 50% at 1:1 RR

Trail remainder with parabolic SAR

Close all positions by 3:30 PM ET

### Key Optimization Parameters:

EMA periods (15-25)

Bollinger Band width multiplier (1.8-2.2)

RSI thresholds (±5)

ATR multiple for stops (0.8-1.5)

### Backtesting Tips:

Test on 15-min charts across different market phases

Prioritize trades with multiple confluence factors

Use walk-forward optimization

Focus on 10 AM - 2 PM ET window for best results

### Critical Notes:

Requires strict discipline in execution

Needs regular parameter re-calibration

Performs best in liquid instruments (ES, NQ, CL)

Combine with sector correlation filters for equity trading

This strategy achieves ~65-75% win rate by:

- Capturing trend continuations with favorable risk/reward

- Exploiting overextensions in ranges

- Using volatility-adjusted position sizing

- Implementing adaptive trade management



## Code:

In [3]:
import backtrader as bt
import datetime
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

### Deepseek Code w/ Notes + My Advancements

In [18]:
class AdvancedHybridStrategy(bt.Strategy):
    params = (
        ('ema_fast', 20),
        ('ema_slow', 50),
        ('bb_period', 20),
        ('bb_dev', 2),
        ('rsi_period', 14),
        ('atr_period', 14),
        ('risk_pct', 1.5),
        ('max_positions', 3),
    )

    def __init__(self):
        # Trend indicators
        self.ema20 = bt.indicators.EMA(period=self.p.ema_fast)
        self.ema50 = bt.indicators.EMA(period=self.p.ema_slow)
        
        # Bollinger Bands for range detection
        self.bb = bt.indicators.BollingerBands(period=self.p.bb_period, devfactor=self.p.bb_dev)
        
        # Dual RSI systems; should probably use RSI SMA since BBs use SMA
        self.rsi_trend = bt.indicators.RSI_EMA(self.data.close, period=self.p.rsi_period)
        self.rsi_safe = bt.indicators.SmoothedMovingAverage(bt.indicators.RSI(self.data.close, period=2), period=3)
        
        # Volatility measure
        self.atr = bt.indicators.ATR(period=self.p.atr_period)
        
        # Trade tracking; do this later
        self.active_trades = []
        self.order = None

        self.dataclose = self.datas[0]


    def market_condition(self):
        # Determine market condition - trending or consolidating
        # if the close price is above fast ema and fast ema is above slow ema -> up trend
        # if the close price is below fast ema and fast ema is below slow ema -> down trend
        trend_cond = (self.data.close[0] > self.ema20[0] > self.ema50[0]) or \
                     (self.data.close[0] < self.ema20[0] < self.ema50[0])

        # if the width of the bands is divided by the average price, this gives the width of the bands
        # per unit change in average price. If this value is small, below some arbitrary number like 0.5,
        # then it means that even as the average price increases, the bands' wdith only changes by a small amount
        # thus, this means the volatillity is low
        # the next condition is to confirm volatility using ATR so we take the last completed candle and find 
        # the range and then see if it is smaller than the true volatility (ATR) times some random constant, 1.2

        #Note: 1.2 and 0.5 can be changed based on backtest results
        range_cond = (self.bb.top[0] - self.bb.bot[0]) / self.bb.mid[0] < 0.5 and \
                     self.data.high[-1] - self.data.low[-1] < 1.2 * self.atr[0]
        
        return 'trend' if trend_cond else 'range' if range_cond else 'neutral'

    def next(self):
        
        #log the closing price
        self.log('Close, %.2f' % self.dataclose[0])

        # commented out: "or not self.is_trading_time()
        # if an order is pending or its not trading time or we have reached the maximum number of active trades, do nothing
        if self.order or len(self.active_trades) >= self.params.max_positions:
            return

        # determine market condition
        market_condition = self.market_condition()

        # determine position_size (note: in the future using the market_conditon as a param in position_size might be better)
        position_size = self.calculate_position_size()

        if market_condition == 'trend':
            self.process_trend_entries(position_size)
        elif market_condition == 'range':
            self.process_range_entries(position_size)

    def calculate_position_size(self):
        # Risk 1.5% per trade based on ATR stop
        risk_amount = self.broker.getvalue() * (self.params.risk_pct / 100)
        atr_risk = self.atr[0] * 1.5  # 1.5x ATR stop
        size = risk_amount / (atr_risk * self.data.close[0])
        return int(size)

    def process_trend_entries(self, size):
        # Long conditions
        long_signal = (
            self.data.close[0] > self.ema20[0] and
            self.rsi_trend[0] > 40 and
            self.data.volume[0] > 1.5 * bt.indicators.SMA(self.data.volume, period=20)[0]
        )
        
        # Short conditions
        short_signal = (
            self.data.close[0] < self.ema20[0] and
            self.rsi_trend[0] < 60 and
            self.data.volume[0] > 1.5 * bt.indicators.SMA(self.data.volume, period=20)[0]
        )

        if long_signal and not self.position:
            self.execute_trade(size, is_long=True)
        elif short_signal and not self.position:
            self.execute_trade(size, is_long=False)

    def process_range_entries(self, size):
        # Long conditions
        long_signal = (
            self.data.close[0] <= self.bb.bot[0] and
            self.rsi_safe[0] < 35 and
            self.data.close[0] > self.data.open[0]  # Bullish reversal candle
        )
        
        # Short conditions
        short_signal = (
            self.data.close[0] >= self.bb.top[0] and
            self.rsi_safe[0] > 65 and
            self.data.close[0] < self.data.open[0]  # Bearish reversal candle
        )

        if long_signal and not self.position:
            self.execute_trade(size, is_long=True)
        elif short_signal and not self.position:
            self.execute_trade(size, is_long=False)

    def execute_trade(self, size, is_long):
        price = self.data.close[0]
        stop_price = price - (1.5 * self.atr[0]) if is_long else price + (1.5 * self.atr[0])
        take_profit = price + (2.5 * self.atr[0]) if is_long else price - (2.5 * self.atr[0])

        
        # Create bracket order with take_profit and stop_loss using the calculated values
        if is_long:
            
            self.order = self.buy_bracket(size=size, exectype=bt.Order.Market, stopprice=stop_price, 
                                          price=price, limitprice=take_profit, oargs={},)
        else:
            self.order = self.sell_bracket(size=size, exectype=bt.Order.Market, stopprice=stop_price, 
                                           price=price, limitprice=take_profit, oargs={},)
        
        self.active_trades.append(self.order)

    def notify_trade(self, trade):
        if trade.isclosed:
            self.active_trades = [t for t in self.active_trades if t.status not in [bt.Order.Completed, bt.Order.Canceled]]

    def stop(self):
        self.log('Final Portfolio Value: %.2f' % self.broker.getvalue())

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    #def is_trading_time(self):
        # Avoid first/last 30 minutes due to very unpredictable market conditions (adjust for your market)
        #time = self.data.datetime.time()
        #return (datetime.time(9, 30) < time < datetime.time(15, 30))


### Backtesting engine

In [17]:
cerebro = bt.Cerebro()

cerebro.addstrategy(AdvancedHybridStrategy)

cerebro.broker.setcash(1000000.0)


modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
modpath, _ = modpath.split(r'/venv')
datapath = os.path.join(modpath, 'data_folder/nvda-1999-2014.txt')

data = bt.feeds.YahooFinanceCSVData(dataname=datapath, fromdate=datetime.datetime(1999, 1, 25), todate=datetime.datetime(2010, 4, 20),
                                    reverse=False)

cerebro.broker.setcommission(commission=0.001)

cerebro.adddata(data)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()


Starting Portfolio Value: 1000000.00
1999-04-06, Close, 1.49


IndexError: array index out of range

In [13]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Final Portfolio Value: 1000000.00
